# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import os
import numpy as np
import math
import tensorflow as tf

from time import time
from datetime import datetime
from keras.callbacks import TensorBoard

from data_handler_MNIST import MNISTDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions
from CMAPSAuxFunctions import TrainValTensorBoard

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0
l1_lambda_regularization = 0
def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_shape=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_shape=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
#     model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
#                     kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
#                     name='fc3'))
    model.add(Dense(10, activation='softmax',kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))



    
    return model

# Create Tunable Model

In [3]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0,beta_1=0.5)
    
    lossFunction = "categorical_crossentropy"
    metrics = ["accuracy"]
    model = None

    #Create and compile the models
    model = RULmodel_SN_5(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [4]:
# num_features = len(selected_features)

shape = (784,)
dHandler_mnist = MNISTDataHandler()
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelMNIST_SN_1', model, lib_type='keras', data_handler=dHandler_mnist)


# Load Data

In [5]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.load_data(verbose=1, cross_validation_ratio=0.8)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data. Cros-Validation ratio 0.8
170500096/170498071 [==============================] - 45s 0us/step
Printing shapes

Training data (X, y)
(9999, 3072)
(9999, 10)
Cross-Validation data (X, y)
(40001, 3072)
(40001, 10)
Testing data (X, y)
(10000, 3072)
(10000, 10)
Printing first 5 elements

Training data (X, y)
[[0.01568628 0.02745098 0.05490196 ... 0.18431373 0.2        0.16470589]
 [0.67058825 0.5882353  0.654902   ... 0.75686276 0.6745098  0.7529412 ]
 [0.45882353 0.5058824  0.5137255  ... 0.4        0.37254903 0.3882353 ]
 [0.01568628 0.01960784 0.01960784 ... 0.08235294 0.17254902 0.34509805]
 [0.47843137 0.5568628  0.78431374 ... 0.78431374 0.7294118  0.62352943]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]
Cross-Validation data (X, y)
[[0.10980392 0.10980392 0.10980392 ... 0.2        0.2        0.2       ]
 [0.18431373 0.19215687 0.1882353  ... 0.490196

/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# Train the model and test some Tunable Model functionalities

In [7]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = (3072,)


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')

# now = datetime.now()
# tensorboard = TrainValTensorBoard(log_dir="./logs/"+now.strftime("%Y%m%d-%H%M%S")+ "/",write_graph=False)

tModel.epochs = 20
tModel.train_model(learningRate_scheduler=lrate, verbose=1)




training with cv
Train on 9999 samples, validate on 40001 samples
Epoch 1/20
9999/9999 [==============================] - 1s 108us/step - loss: 2.2487 - acc: 0.1559 - val_loss: 2.1617 - val_acc: 0.2111
Epoch 2/20
9999/9999 [==============================] - 0s 45us/step - loss: 2.1295 - acc: 0.2117 - val_loss: 2.0923 - val_acc: 0.2321
Epoch 3/20
9999/9999 [==============================] - 0s 47us/step - loss: 2.0752 - acc: 0.2329 - val_loss: 2.0360 - val_acc: 0.2529
Epoch 4/20
9999/9999 [==============================] - 0s 44us/step - loss: 2.0345 - acc: 0.2465 - val_loss: 2.0043 - val_acc: 0.2641
Epoch 5/20
9999/9999 [==============================] - 0s 49us/step - loss: 1.9856 - acc: 0.2728 - val_loss: 1.9917 - val_acc: 0.2676
Epoch 6/20
9999/9999 [==============================] - 0s 48us/step - loss: 1.9622 - acc: 0.2800 - val_loss: 1.9442 - val_acc: 0.2922
Epoch 7/20
9999/9999 [==============================] - 0s 45us/step - loss: 1.9438 - acc: 0.2936 - val_loss: 1.9364 - val_

In [8]:
tModel.evaluate_model(cross_validation=True)

cScores = tModel.scores
print(cScores)

40001/40001 [==============================] - 1s 17us/step
{'loss': 1.88600313084438, 'score_1': 0.31419214519637007}


In [9]:
tModel.evaluate_model(cross_validation=False)

cScores = tModel.scores
print(cScores)

10000/10000 [==============================] - 0s 18us/step
{'loss': 1.8851857538223267, 'score_1': 0.3154}
